# Program Sudoku Solver:
- jedná se o program, který dokáže vyluštit zadané sudoku a to nejen standardní, ale i specializované typy (popis viz níže)
- použitá databáze a typy pochází z MAČR (Mistrovství Akademiků ČR) z roku 2020
- odkaz na úlohy: http://sudokualogika.cz/sites/default/files/macr2020_PB.pdf

# Popis tříd:
- sudoku solver obsahuje hlavní třídu "Sudoku", která obsahuje řadu menších tříd
- podtřída "Uloha" je určena k načtení zadání ze souboru a základnímu zpracování těchto vstupních dat
- podtřídy "Policko" se objevují v tabulce 9x9 a udržují informace jako - zda je konkrétní políčko již vyplněno a jakým číslem, jaká čísla lze nebo nelze vepsat atd.
- podtřída "Dalsi" určuje, jak bude hlavní program postupovat při luštění sudoku, zda bude vepisovat, měnit či mazat čísla atd.
- podtřídy "Krok" se budou vyskytovat jako prvky dynamického listu a budou nést informace o průběhu vyplňování sudoku, slouží zejména pro funkci rollbacku, když se dostaneme do slepé větve
- podtřídy "Resitel_SpecialniTyp":
    - používají se pro luštění různých druhů sudoku
    - každá obsahuje funkci "banuj", která se vyvolává při zapsání čísla a určuje, do jakých jiných polí již nemohou být některá čísla zapsána
    - jejich základem je pole "policka", které obsahuje zpravidla 2 informace:
        - souřadnice sloužící jako odkaz na jiné políčko
        - dodatečné informace (např. zda je mezi políčky nerovnost (viz níže typy sudoku))
    - pole "policka" se tvori pri iniciaci samotneho objektu, pouziva se ve funkci "banuj"

# Popis logiky programu:
- dle typu sudoku (typy lze libovolně kombinovat) si hlavní třída "Sudoku" vytvoří list s podtřídami "Resitel_SpecialniTyp", které bude později volat
- "Sudoku" si udržuje informace o tom, kolik různých čísel lze do jakého pole zapsat
- po každém kroku se rozhodujeme, jak budeme dále postupovat:
    - pokud existuje prázdné pole, kam nelze zapsat žádné číslo, narazili jsme na slepou větev a musíme se vrátit:
        - zkusíme změnit posledně zapsané číslo, pokud nelze, pak jej smažeme
    - pokud jsme mazali nesmí být další krok zapsání čísla, jinak se zacyklíme
    - ve zbývajících případech zapíšeme další číslo
- je navržena heuristika, kdy nejprve vyplňujeme ta pole, do kterých lze zapsat nejnižší počet čísel a v případě slepých větví tak nemusíme procházet tolik možností
- tato heuristika je upravena pomocí prvků "hodnota_bonus" v podtřídách "Policko", je to proto, že u specializovaných typů sudoku chceme vyplňovat pole se speciálními vlastnostmi nebo informacemi jako první

# Popis různých druhů sudoku:
- vždy platí, že čísla v řádcích ani sloupcích se nesmí opakovat
- Basic - existuje 9 oblatí 3x3, ve kterých se čísla nesmí opakovat
- Diagonal - na 2 hlavních diagonálách se nesmí čísla opakovat
- Windoku - existují 4 další oblasti ([2,2]:[4-4], [2,6]:[4,8], [6,2]:[8,4], [6,6]:[8,8]), ve kterých se čísla nesmí opakovat
- Region - existují nepravidelné oblasti, ve kterých se nesmí čísla opakovat (buďto nahrazuje nebo doplňuje typ Basic)
- Suda - na podbarvených polích mohou být pouze sudá čísla
- Licha - na podbarvených polích mohou být pouze lichá čísla
- AntiJezdec - lze-li z jednoho pole na druhé skočit šachovým jezdcem, musí tato pole obsahovat různá čísla
- Pevnost - sousedí-li podbarvené pole s nepodbarveným, pak číslo v podbarveném poli musí být větší
- Nerovnost - mezi některými sousedícími poli jsou vyznačeny nerovnosti, tyto nerovnosti musí být dodrženy
- Teplomer - v každém vyznačeném teploměru (který zasahuje přes několik polí) musí čísla růst od baňky ke špičce
- Sousledna - symbolem jsou vyznačeny všechny dvojice sousedících polí, kde se čísla liší o 1
- Rimska - symboly 'V' a 'X' jsou vyznačeny všechny dvojice sousedících polí, jejichž součet je 5 nebo 10
- Rozdil - některé dvojice sousedících polí mají zadaný svůj rozdíl
- KdeJe9 - některá pole jsou podbarvena šipkami, tyto šipky udávají, kde v daném řádku či sloupci se nachází číslo 9, zároveň číslo v tomto podbarveném poli udává, o kolik polí je 9 od tohoto pole vzdálena

# Postup přidání nové třídy "Resitel_SpecialniTyp":
- Sudoku Solver je jako takový v podstatě hotov, zbývá jej rozšířit o nové typy specializovaných sudoku, což vlastně znamená vytvořit nové třídy "Resitel_SpecialniTyp" a drobná úprava kódu:
    - do třídy "Uloha" přidat novou položku do knihovny "mozne_objekty"
    - definovat samotnou třídu, musí obsahovat funkce "napis_zadani" a "banuj"
    - ve třídě "Sudoku" přidat do funkce "nacti_resitele"

In [1]:
import copy
import csv

In [2]:
class Policko:
    def __init__(self, rozmer_cisla):
        self.cislo = 0
        self.bany = []
        for i in range(rozmer_cisla):
            self.bany.append(False)
        self.bany_pocet = 0
        self.hodnota = 0
        self.hodnota_bonus = 0

In [3]:
class Dalsi:
    def __init__(self):
        self.radek = 1
        self.sloupec = 1
        self.hodnota = 0
        self.indikator = 0
        self.error = False

In [4]:
class Krok_Ban:
    def __init__(self, radek, sloupec, cislo):
        self.radek = radek
        self.sloupec = sloupec
        self.cislo = cislo

In [5]:
class Krok:
    def __init__(self, radek, sloupec):
        self.radek = radek
        self.sloupec = sloupec
        self.mozna_cisla = []
        self.cislo_poradi = 0
        self.bany = []

In [6]:
class Uloha:
    def __init__(self, soubor):
        self.rozmer_cisla = 9
        self.rozmer_mrizka = 9
        # objekt si otevre soubor, ocekava na prvnim radku seznam typu sudoku
        self.soubor = open(soubor)
        self.soubor_obsah = list(csv.reader(self.soubor))
        self.typ = self.soubor_obsah[0][0].split(';')
        # dle typu sudoku a klicu nasledujici klihovny, objekt ocekava prislusne hodnoty knihovny v souboru v prvnim sloupci
            # pokud se v souboru vsechny polozky nevyskytuji, nevadi, objekt je iniciuje jako prazdna pole
            # pokud se v souboru vyskytuji nadbytecne polozky, objekt je ignoruje
        self.mozne_objekty = {'':['cisla'],
                              'basic':[],
                              'diagonal':[],
                              'windoku':[],
                              'region':['region'],
                              'suda':['suda'],
                              'licha':['licha'],
                              'antijezdec':[],
                              'pevnost':['pevnost'],
                              'nerovnost':['nerovnost_radek', 'nerovnost_sloupec'],
                              'teplomer':['teplomer_1', 'teplomer_2'],
                              'sousledna':['sousledna_radek', 'sousledna_sloupec'],
                              'rimska':['rimska_radek', 'rimska_sloupec'],
                              'rozdil':['rozdil_radek', 'rozdil_sloupec'],
                              'kdeje9':['kdeje9']}
        # vytvorime pomocnou knihovnu, kde si vyselektujeme jen ty objekty, ktere potrebujeme pro dany typ sudoku
        self.mozne_objekty_map = {'cisla':[]}
        for typ in self.typ:
            for objekt in self.mozne_objekty[typ]:
                self.mozne_objekty_map[objekt] = []
        # nyni si iniciujeme samotnou knihovnu 'objekty', kterou budeme v dalsim postupu pouzivat
        self.objekty = {}
        for objekt in self.mozne_objekty_map.keys():
            self.objekty[objekt] = []
            for i in range(self.rozmer_mrizka):
                self.objekty[objekt].append([])
                for j in range(self.rozmer_mrizka):
                    self.objekty[objekt][-1].append('')
        # nyni je potreba poznacit si, kde v souboru se vyskytuji potrebne objekty a zapsat si do pomocne knihovny souradnice
        self.pomocny_int = -1
        self.pomocny_str = ''
        for i in range(len(self.soubor_obsah)):
            if self.soubor_obsah[i][0].split(';')[0] in self.mozne_objekty_map.keys():
                if self.pomocny_int != -1:
                    self.mozne_objekty_map[self.pomocny_str].append(i)
                self.pomocny_int = i
                self.pomocny_str = self.soubor_obsah[i][0].split(';')[0]
                self.mozne_objekty_map[self.pomocny_str].append(self.pomocny_int)
        self.mozne_objekty_map[self.pomocny_str].append(len(self.soubor_obsah))
        # nyni do nasi hlavni knihovny vepiseme informace ze vstupniho souboru
        for objekt in self.mozne_objekty_map.keys():
            if len(self.mozne_objekty_map[objekt]) > 0:
                for i in range(min(self.rozmer_mrizka, self.mozne_objekty_map[objekt][1] - self.mozne_objekty_map[objekt][0] - 1)):
                    for j in range(min(self.rozmer_mrizka, len(self.soubor_obsah[0][0].split(';')))):
                        self.objekty[objekt][i][j] = self.soubor_obsah[self.mozne_objekty_map[objekt][0] + i + 1][0].split(';')[j]
        self.soubor.close()
        self.soubor = ''

In [7]:
# zde si definujeme nekolik funkci na vypsani zadani sudoku
# rozlisujeme 4 druhy (v tomto poradi i funkce nize):
    # vypsani policek eventualne informaci, ktere se na polickach nachazeji
    # vypsani informaci, ktere se nachazeji na hrane dvou sousedicich poli na radku (jako nerovnost, jejich rozdil)
    # vypsani informaci, ktere se nachazeji na hrane dvou sousedicich poli ve sloupci
    # vypsani informaci, ktere se nachazeji v rohu sousedicich poli (jako jejich soucet, vzestupne poradi dle smeru hodinovych rucicek)
def print_policka(policka, rozmer_mrizka, popis):
    print(popis)
    for i in range(rozmer_mrizka):
        pomocny_string = ''
        if i % 3 == 0:
            print('-------------')
        for j in range(rozmer_mrizka):
            if j % 3 == 0:
                pomocny_string += '|'
            if policka[i][j] == '':
                pomocny_string += ' '
            else:
                pomocny_string += policka[i][j]
        pomocny_string += '|'
        print(pomocny_string)
    print('-------------')
    print()
def print_radky(policka, rozmer_mrizka, popis):
    print(popis)
    for i in range(rozmer_mrizka):
        if i % 3 == 0:
            print('----------')
        pomocny_string = '|'
        for j in range(rozmer_mrizka - 1):
            if policka[i][j] != '':
                pomocny_string += policka[i][j]
            else:
                if j % 3 == 2:
                    pomocny_string += '|'
                else:
                    pomocny_string += ' '
        pomocny_string += '|'
        print(pomocny_string)
    print('----------')
    print()
def print_sloupce(policka, rozmer_mrizka, popis):
    print(popis)
    print('-------------')
    for i in range(rozmer_mrizka - 1):
        pomocny_string = ''
        for j in range(rozmer_mrizka):
            if j % 3 == 0:
                pomocny_string += '|'
            if policka[i][j] != '':
                pomocny_string += policka[i][j]
            else:
                if i % 3 == 2:
                    pomocny_string += '-'
                else:
                    pomocny_string += ' '
        pomocny_string += '|'
        print(pomocny_string)
    print('-------------')
    print()
def print_rohy(policka, rozmer_mrizka, popis):
    print(popis)
    print('----------')
    for i in range(rozmer_mrizka):
        pomocny_string = '|'
        for j in range(rozmer_mrizka):
            if policka[i][j] != '':
                pomocny_string += policka[i][j]
            else:
                if i % 3 == 2:
                    pomocny_string += '-'
                else:
                    if j % 3 == 2:
                        pomocny_string += '|'
                    else:
                        pomocny_string += ' '
        pomocny_string += '|'
        print(pomocny_string)
    print('----------')
    print()

Následují třídy "Resitel_SpecialniTyp":
- všechny obsahují funkce "napis_zadani" a "banuj"
- inicializační funkce zpravidla vytvoří pole "policka" (velikosti 9x9):
    - toto pole je základem pro funkci "banuj"
    - každý prvek tohoto pole je list, který je tvořen dvojicemi či trojicemi
    - první dvě položky každé této dvojice či trojice jsou souřadnice odkazující na jiné políčko (např. jsou-li pole ve stejné oblasti, mají-li mezi sebou znaménko)
    - třetí položka (pokud existuje) uchovává informaci o jejich vzájemné propojenosti (např. které je větší, jaký je jejich rozdíl)
- funkce "banuj" vždy pro zadaný řádek a sloupec projde příslušný list pole "policka" a provede příslušné kroky (zpravidla banuje některá čísla v políčku, na které se odkazujeme)

In [8]:
# zakladni trida pouzivana vzdy
# hlida, aby na radku ani sloupci nebyly dve stejna cisla
class Resitel_Sudoku:
    def __init__(self, sudoku):
        pass
    def napis_zadani(self, sudoku):
        print('Sudoku zadani:')
        pomocny_string = 'Typ: '
        for typ in sudoku.uloha.typ:
            pomocny_string = pomocny_string + typ + ' '
        print_policka(sudoku.uloha.objekty['cisla'], sudoku.rozmer_mrizka, pomocny_string)
    def banuj(self, sudoku, radek, sloupec, cislo):
        # pri zapsani daneho cisla banujeme toto cislo ze vsech poli daneho radku a sloupce
        for i in range(sudoku.rozmer_mrizka):
            sudoku.banuj_policko(radek, i + 1, cislo)
            sudoku.banuj_policko(i + 1, sloupec, cislo)

In [9]:
# pravidlo - existuje 9 oblasti 3x3, ve kterych se cisla nesmi opakovat
# vstup - neni
class Resitel_Basic:
    def __init__(self, sudoku):
        # vytvorime si pomocne pole s danymi oblastmi
        self.policka_pomocna = []
        self.policka_pomocna.append(((1,1),(1,2),(1,3),(2,1),(2,2),(2,3),(3,1),(3,2),(3,3)))
        self.policka_pomocna.append(((1,4),(1,5),(1,6),(2,4),(2,5),(2,6),(3,4),(3,5),(3,6)))
        self.policka_pomocna.append(((1,7),(1,8),(1,9),(2,7),(2,8),(2,9),(3,7),(3,8),(3,9)))
        self.policka_pomocna.append(((4,1),(4,2),(4,3),(5,1),(5,2),(5,3),(6,1),(6,2),(6,3)))
        self.policka_pomocna.append(((4,4),(4,5),(4,6),(5,4),(5,5),(5,6),(6,4),(6,5),(6,6)))
        self.policka_pomocna.append(((4,7),(4,8),(4,9),(5,7),(5,8),(5,9),(6,7),(6,8),(6,9)))
        self.policka_pomocna.append(((7,1),(7,2),(7,3),(8,1),(8,2),(8,3),(9,1),(9,2),(9,3)))
        self.policka_pomocna.append(((7,4),(7,5),(7,6),(8,4),(8,5),(8,6),(9,4),(9,5),(9,6)))
        self.policka_pomocna.append(((7,7),(7,8),(7,9),(8,7),(8,8),(8,9),(9,7),(9,8),(9,9)))
        # abychom ve funkci 'banuj' nemuseli vsechny polozky predchoziho pole tupe prochazet, vytvorime si nove pole 9x9:
            # kazde policko bude obsahovat odkazy na jina policka, ktera se nachazeji ve stejne oblasti a musi tedy byt banovana
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for policko_pomocne in self.policka_pomocna:
            for i in range(len(policko_pomocne)):
                for j in range(i + 1, len(policko_pomocne)):
                    self.policka[policko_pomocne[i][0] - 1][policko_pomocne[i][1] - 1].append(policko_pomocne[j])
                    self.policka[policko_pomocne[j][0] - 1][policko_pomocne[j][1] - 1].append(policko_pomocne[i])
    def napis_zadani(self, sudoku):
        pass
    def banuj(self, sudoku, radek, sloupec, cislo):
        # banujeme vsechna policka, ktera se nachazela ve stejne oblasti
        for policko in self.policka[radek - 1][sloupec - 1]:
            sudoku.banuj_policko(policko[0], policko[1], cislo)

In [10]:
# pravidlo - na dvou hlavnich diagonalach se cisla nesmi opakovat
# vstup - neni
# naprosto stejna logika jako 'Resitel_Basic'
class Resitel_Diagonal:
    def __init__(self, sudoku):
        self.hodnota_bonus = 100
        self.policka_pomocna = []
        self.policka_pomocna.append(((1,1),(2,2),(3,3),(4,4),(5,5),(6,6),(7,7),(8,8),(9,9)))
        self.policka_pomocna.append(((1,9),(2,8),(3,7),(4,6),(5,5),(6,4),(7,3),(8,2),(9,1)))
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for policko_pomocne in self.policka_pomocna:
            for i in range(len(policko_pomocne)):
                # cislum na diagonale dame maly bonus, abychom je zacali vyplnovat prednostneji nez ostatni
                sudoku.policka[policko_pomocne[i][0] - 1][policko_pomocne[i][1] - 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[policko_pomocne[i][0] - 1][policko_pomocne[i][1] - 1].hodnota_bonus)
                for j in range(i + 1, len(policko_pomocne)):
                    self.policka[policko_pomocne[i][0] - 1][policko_pomocne[i][1] - 1].append(policko_pomocne[j])
                    self.policka[policko_pomocne[j][0] - 1][policko_pomocne[j][1] - 1].append(policko_pomocne[i])
    def napis_zadani(self, sudoku):
        pass
    def banuj(self, sudoku, radek, sloupec, cislo):
        for policko in self.policka[radek - 1][sloupec - 1]:
            sudoku.banuj_policko(policko[0], policko[1], cislo)

In [11]:
# pravidlo - ve ctyrech specialnich oblastech se cisla nesmi opakovat
# vstup - neni
# naprosto stejna logika jako 'Resitel_Basic'
class Resitel_Windoku:
    def __init__(self, sudoku):
        self.hodnota_bonus = 100
        self.policka_pomocna = []
        self.policka_pomocna.append(((2,2),(2,3),(2,4),(3,2),(3,3),(3,4),(4,2),(4,3),(4,4)))
        self.policka_pomocna.append(((2,6),(2,7),(2,8),(3,6),(3,7),(3,8),(4,6),(4,7),(4,8)))
        self.policka_pomocna.append(((6,2),(6,3),(6,4),(7,2),(7,3),(7,4),(8,2),(8,3),(8,4)))
        self.policka_pomocna.append(((6,6),(6,7),(6,8),(7,6),(7,7),(7,8),(8,6),(8,7),(8,8)))
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for policko_pomocne in self.policka_pomocna:
            for i in range(len(policko_pomocne)):
                sudoku.policka[policko_pomocne[i][0] - 1][policko_pomocne[i][1] - 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[policko_pomocne[i][0] - 1][policko_pomocne[i][1] - 1].hodnota_bonus)
                for j in range(i + 1, len(policko_pomocne)):
                    self.policka[policko_pomocne[i][0] - 1][policko_pomocne[i][1] - 1].append(policko_pomocne[j])
                    self.policka[policko_pomocne[j][0] - 1][policko_pomocne[j][1] - 1].append(policko_pomocne[i])
    def napis_zadani(self, sudoku):
        pass
    def banuj(self, sudoku, radek, sloupec, cislo):
        for policko in self.policka[radek - 1][sloupec - 1]:
            sudoku.banuj_policko(policko[0], policko[1], cislo)

In [12]:
# pravidlo - ve specialne vyznacenych regionech se cisla nesmi opakovat
# vstup - tabulka 9x9, kde kazdy region ma svuj string, ktery je vepsan do kazdeho pole regionu
# naprosto stejna logika jako 'Resitel_Basic', jen se vstupni regiony musi nacist ze souboru
class Resitel_Region:
    def __init__(self, sudoku):
        self.hodnota_bonus = 100
        self.policka_pomocna = {}
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka):
                if sudoku.uloha.objekty['region'][i][j] != '':
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    if sudoku.uloha.objekty['region'][i][j] in self.policka_pomocna.keys():
                        self.policka_pomocna[sudoku.uloha.objekty['region'][i][j]].append((i + 1, j + 1))
                    else:
                        self.policka_pomocna[sudoku.uloha.objekty['region'][i][j]] = [(i + 1, j + 1)]
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for policko_pomocne in self.policka_pomocna.values():
            for i in range(len(policko_pomocne)):
                for j in range(i + 1, len(policko_pomocne)):
                    self.policka[policko_pomocne[i][0] - 1][policko_pomocne[i][1] - 1].append(policko_pomocne[j])
                    self.policka[policko_pomocne[j][0] - 1][policko_pomocne[j][1] - 1].append(policko_pomocne[i])
    def napis_zadani(self, sudoku):
        print_policka(sudoku.uloha.objekty['region'], sudoku.rozmer_mrizka, 'Region:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        for policko in self.policka[radek - 1][sloupec - 1]:
            sudoku.banuj_policko(policko[0], policko[1], cislo)

In [13]:
# pravidlo - na vyznacenych polich se nachazi pouze suda cisla
# vstup - tabulka 9x9, pole, ktera obsahuji pouze suda cisla, oznacena 'X'
class Resitel_Suda:
    def __init__(self, sudoku):
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka):
                # hned na pocatku si zabanujeme licha cisla z oznacenych poli
                if sudoku.uloha.objekty['suda'][i][j] == 'X':
                    for k in range(0, sudoku.rozmer_cisla, 2):
                        sudoku.banuj_policko(i + 1, j + 1, k + 1)
    def napis_zadani(self, sudoku):
        print_policka(sudoku.uloha.objekty['suda'], sudoku.rozmer_mrizka, 'Suda:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        # tento objekt pri lusteni jiz nepomaha, svou praci odvedl na zacatku
        pass

In [14]:
# pravidlo - na vyznacenych polich se nachazi pouze licha cisla
# vstup - tabulka 9x9, pole, ktera obsahuji pouze licha cisla, oznacena 'X'
# naprosto stejna logika jako 'Resitel_Suda'
class Resitel_Licha:
    def __init__(self, sudoku):
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka):
                if sudoku.uloha.objekty['licha'][i][j] == 'X':
                    for k in range(1, sudoku.rozmer_cisla, 2):
                        sudoku.banuj_policko(i + 1, j + 1, k + 1)
    def napis_zadani(self, sudoku):
        print_policka(sudoku.uloha.objekty['licha'], sudoku.rozmer_mrizka, 'Licha:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        pass

In [15]:
# pravidlo - lze-li z jednoho pole na druhe skocit sachovym jezdcem, pak tato pole musi obsahovat ruzna cisla
# vstup - neni
class Resitel_AntiJezdec:
    def __init__(self, sudoku):
        self.hodnota_bonus = 10
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
                # kazdemu policku vytvorime seznam policek, ne ktere se da skocit sachovym jezdcem
                # je potreba davat pozor, aby nam nepretekl range pole
                if i + 2 < sudoku.rozmer_mrizka and j + 1 < sudoku.rozmer_mrizka:
                    self.policka[-1][-1].append((i + 3, j + 2))
                if i + 2 < sudoku.rozmer_mrizka and j > 0:
                    self.policka[-1][-1].append((i + 3, j))
                if i + 1 < sudoku.rozmer_mrizka and j + 2 < sudoku.rozmer_mrizka:
                    self.policka[-1][-1].append((i + 2, j + 3))
                if i + 1 < sudoku.rozmer_mrizka and j > 1:
                    self.policka[-1][-1].append((i + 2, j - 1))
                if i > 0 and j + 2 < sudoku.rozmer_mrizka:
                    self.policka[-1][-1].append((i, j + 3))
                if i > 0 and j > 1:
                    self.policka[-1][-1].append((i, j - 1))
                if i > 1 and j + 1 < sudoku.rozmer_mrizka:
                    self.policka[-1][-1].append((i - 1, j + 2))
                if i > 1 and j > 0:
                    self.policka[-1][-1].append((i - 1, j))
                # policko dostane drobny bonus dle toho, na kolik jinych poli se da jezdcem skocit
                sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus * len(self.policka[-1][-1]), sudoku.policka[i][j].hodnota_bonus)
    def napis_zadani(self, sudoku):
        pass
    def banuj(self, sudoku, radek, sloupec, cislo):
        # banujeme pole, na ktere se da skocit jezdcem
        for policko in self.policka[radek - 1][sloupec - 1]:
            sudoku.banuj_policko(policko[0], policko[1], cislo)

In [16]:
# pravidlo - pokud podbarvene pole (pevnost) sousedi s nepodbarvenym, pak cislo v podbarvenem poli je vetsi
# vstup - tabulka 9x9, pole pevnosti oznacena 'X'
class Resitel_Pevnost:
    def __init__(self, sudoku):
        self.hodnota_bonus = 500
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka):
                # pokud narazime na policko pevnosti
                if sudoku.uloha.objekty['pevnost'][i][j] == 'X':
                    # pokud sousedici pole neni pevnost
                    if i > 0 and sudoku.uloha.objekty['pevnost'][i - 1][j] == '':
                        # zapiseme teto dvojici policek vzajemny odkaz na sebe
                        # treti polozka po souradnicich udava, ktere z techto poli je pevnost a tedy vetsi (1 jsem pevnost, -1 nejsem pevnost)
                        self.policka[i][j].append((i, j + 1, 1))
                        self.policka[i - 1][j].append((i + 1, j + 1, -1))
                        # zaroven vime, ze pevnost nemuze byt 1 a pole vedle pevnosti 9
                        sudoku.banuj_policko(i + 1, j + 1, 1)
                        sudoku.banuj_policko(i, j + 1, sudoku.rozmer_cisla)
                        # obema polim pripiseme pomerne znacny bonus
                        sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                        sudoku.policka[i - 1][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i - 1][j].hodnota_bonus)
                    if i + 1 < sudoku.rozmer_mrizka and sudoku.uloha.objekty['pevnost'][i + 1][j] == '':
                        self.policka[i][j].append((i + 2, j + 1, 1))
                        self.policka[i + 1][j].append((i + 1, j + 1, -1))
                        sudoku.banuj_policko(i + 1, j + 1, 1)
                        sudoku.banuj_policko(i + 2, j + 1, sudoku.rozmer_cisla)
                        sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                        sudoku.policka[i + 1][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i + 1][j].hodnota_bonus)
                    if j > 0 and sudoku.uloha.objekty['pevnost'][i][j - 1] == '':
                        self.policka[i][j].append((i + 1, j, 1))
                        self.policka[i][j - 1].append((i + 1, j + 1, -1))
                        sudoku.banuj_policko(i + 1, j + 1, 1)
                        sudoku.banuj_policko(i + 1, j, sudoku.rozmer_cisla)
                        sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                        sudoku.policka[i][j - 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j - 1].hodnota_bonus)
                    if j + 1 < sudoku.rozmer_mrizka and sudoku.uloha.objekty['pevnost'][i][j + 1] == '':
                        self.policka[i][j].append((i + 1, j + 2, 1))
                        self.policka[i][j + 1].append((i + 1, j + 1, -1))
                        sudoku.banuj_policko(i + 1, j + 1, 1)
                        sudoku.banuj_policko(i + 1, j + 2, sudoku.rozmer_cisla)
                        sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                        sudoku.policka[i][j + 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j + 1].hodnota_bonus)
    def napis_zadani(self, sudoku):
        print_policka(sudoku.uloha.objekty['pevnost'], sudoku.rozmer_mrizka, 'Pevnost:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        # projdem odkazy na jina pole
        for policko in self.policka[radek - 1][sloupec - 1]:
            # pokud jsem ja pevnost, pak sousedicimu policku zakazu cisla vetsi nez ja
            if policko[2] == 1:
                for i in range(cislo + 1, sudoku.rozmer_cisla + 1):
                    sudoku.banuj_policko(policko[0], policko[1], i)
            # jinak je sousedici policko mensi nez ja a tedy mu zakazu cisla mensi nez ja
            else:
                for i in range(1, cislo - 1):
                    sudoku.banuj_policko(policko[0], policko[1], i)

In [17]:
# pravidlo - mezi nekterymi sousedicimi policky je nerovnost, ktera musi byt dodrzena
# vstup - tabulky 9x8 a 8x9, je-li mezi dvema sousedicimi policky nerovnost, prislusne pole obsahuje:
    # '<' a '>' pro radky; tabulka 9x8; napr. pokud znak na pozici [2,5] == '<' pak cislo[2,5] < cislo[2,6]
    # 'V' a 'A' pro sloupce; tabulka 8x9; napr. pokud znak na pozici [3,1] == 'V' pak cislo[3,1] > cislo[4,1]
# typ sudoku 'pevnost' je vlastne specialni pripad sudoku typu 'nerovnost', postup tedy bude obdobny jako v 'Resitel_Pevnost'
class Resitel_Nerovnost:
    def __init__(self, sudoku):
        self.hodnota_bonus = 500
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka - 1):
                # pokud narazim na nerovnost
                if sudoku.uloha.objekty['nerovnost_radek'][i][j] == '<':
                    # zapiseme teto dvojici policek vzajemny odkaz na sebe
                    # treti polozka po souradnicich udava, ktere z techto poli je vetsi (1 jsem vetsi, -1 jsem mensi)
                    self.policka[i][j].append((i + 1, j + 2, -1))
                    self.policka[i][j + 1].append((i + 1, j + 1, 1))
                    # obema polim pripiseme pomerne znacny bonus
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i][j + 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j + 1].hodnota_bonus)
                elif sudoku.uloha.objekty['nerovnost_radek'][i][j] == '>':
                    self.policka[i][j].append((i + 1, j + 2, 1))
                    self.policka[i][j + 1].append((i + 1, j + 1, -1))
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i][j + 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j + 1].hodnota_bonus)
        for i in range(sudoku.rozmer_mrizka - 1):
            for j in range(sudoku.rozmer_mrizka):
                if sudoku.uloha.objekty['nerovnost_sloupec'][i][j] == 'A':
                    self.policka[i][j].append((i + 2, j + 1, -1))
                    self.policka[i + 1][j].append((i + 1, j + 1, 1))
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i + 1][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i + 1][j].hodnota_bonus)
                elif sudoku.uloha.objekty['nerovnost_sloupec'][i][j] == 'V':
                    self.policka[i][j].append((i + 2, j + 1, 1))
                    self.policka[i + 1][j].append((i + 1, j + 1, -1))
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i + 1][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i + 1][j].hodnota_bonus)
        # zde je potreba se na chvili zastavit a probrat zbyly postup v teto inicializacni funkci
        # obdobne jako v 'Resitel_Pevnost' muzeme o nekterych polich rici, ze nemohou obsahovat 1 nebo 9 (dle nerovnosti)
        # na rozdil od 'Resitel_Pevnost' se tyto nerovnosti mohou retezit, tedy muze platit, ze:
            # pole A < B < C < D, tedy: A not in [7,8,9], B not in [1,8,9], C not in [1,2,9], D not in [1,2,3]
        # takovychto retezcu muze byt v sudoku mnoho a to ruzne delky (od 2 do 9)
        # nebudeme tedy tyto retezce hledat, ale zvolime jiny, ekvivalentni postup:
            # vytvorime si pomocne pole 'policka_pomocna' 9x9
            # projdeme pole 'policka' a do vsech policek v pomocnem poli 'policka_pomocna', kam lze zapsat cislo 1 (nejsou vetsi nez zadne jine pole) napiseme 1
            # postup opakujeme tentokrat ale s tim rozdilem, ze nerovnosti u poli, ktere obsahuji 1, ignorujeme, a vepisujeme 2
            # postup opakujeme az po cislo 9
            # nyni kazde cislo v poli 'policka_pomocna' udava, jake nejnizsi cislo do nej muze byt zapsano
            # je-li tedy v policku zapsano 3, pak banujeme 1 a 2
        self.policka_pomocna = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka_pomocna.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka_pomocna[-1].append(0)
        for k in range(1, sudoku.rozmer_cisla + 1):
            for i in range(sudoku.rozmer_mrizka):
                for j in range(sudoku.rozmer_mrizka):
                    if self.policka_pomocna[i][j] == 0:
                        self.flag_hodnota = True
                        # projdu vsechna sousedici cisla (respektive ta cisla, kde mezi mnou a jim je nerovnost)
                        for policko in self.policka[i][j]:
                            # pokud jsem vetsi nez jine pole, ktere jeste nema vepsane cislo
                            if policko[2] == 1 and self.policka_pomocna[policko[0] - 1][policko[1] - 1] in [0, k]:
                                self.flag_hodnota = False
                        if self.flag_hodnota:
                            self.policka_pomocna[i][j] = k
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka):
                for k in range(1, self.policka_pomocna[i][j]):
                    sudoku.banuj_policko(i + 1, j + 1, k)
                # zresetujeme si toto pole pro dalsi pouziti
                self.policka_pomocna[i][j] = 0
        # tento postup opakujeme jeste jednou, abychom urcili, jake nejvyssi cisla mohou byt v tech kterych polickach
        for k in range(sudoku.rozmer_cisla, 0, -1):
            for i in range(sudoku.rozmer_mrizka):
                for j in range(sudoku.rozmer_mrizka):
                    if self.policka_pomocna[i][j] == 0:
                        self.flag_hodnota = True
                        for policko in self.policka[i][j]:
                            if policko[2] == -1 and self.policka_pomocna[policko[0] - 1][policko[1] - 1] in [0, k]:
                                self.flag_hodnota = False
                        if self.flag_hodnota:
                            self.policka_pomocna[i][j] = k
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka):
                for k in range(self.policka_pomocna[i][j] + 1, sudoku.rozmer_cisla + 1):
                    sudoku.banuj_policko(i + 1, j + 1, k)
    def napis_zadani(self, sudoku):
        print_radky(sudoku.uloha.objekty['nerovnost_radek'], sudoku.rozmer_mrizka, 'Nerovnost radky:')
        print_sloupce(sudoku.uloha.objekty['nerovnost_sloupec'], sudoku.rozmer_mrizka, 'Nerovnost sloupce:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        for policko in self.policka[radek - 1][sloupec - 1]:
            # jsem-li vetsi, banuji u nej vetsi cisla
            if policko[2] == 1:
                for i in range(cislo + 1, sudoku.rozmer_cisla + 1):
                    sudoku.banuj_policko(policko[0], policko[1], i)
            # jsem-li mensi, banuji u nej mensi cisla
            else:
                for i in range(1, cislo):
                    sudoku.banuj_policko(policko[0], policko[1], i)

In [18]:
# pravidlo - ve vyznacenych teplomerech musi cisla rust od banky ke spicce
# vstup - nekolik tabulek 9x9, kazda z nich muze obsahovat nekolik teplomeru
    # banky teplomeru zacinaji ['1','11','21',...], kazde dalsi cislo teplomeru je o 1 vyssi
    # tabulek s teplomery muze byt vice, protoze napr. teplomery mohou mit spolecnou banku a vstup by musel byt komplikovanejsi
# jedna se o specialni typ znamenkoveho sudoku, postup tedy bude naprosto stejny vyjma nacitani dat
class Resitel_Teplomer:
    def __init__(self, sudoku):
        self.hodnota_bonus = 500
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for teplomer in ['teplomer_1', 'teplomer_2']:
            # vytvorime si pomocnou knihovnu, jeji klice budou cisla od 1 do maximalniho cisla, ktere nalezneme ve vstupnich datech
            self.policka_pomocna = {}
            for i in range(sudoku.rozmer_mrizka):
                for j in range(sudoku.rozmer_mrizka):
                    # pokud nalezneme cast teplomeru
                    if sudoku.uloha.objekty[teplomer][i][j] != '':
                        # zapis si do pomocne knihovny souradnice policka, klic je cislo v danem policku
                        self.policka_pomocna[int(sudoku.uloha.objekty[teplomer][i][j])] = (i + 1, j + 1)
                        # policku pripiseme znacny bonus
                        sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
            # zjisti nejvyssi cislo
            if len(self.policka_pomocna) == 0:
                self.teplomer_max = 0
            else:
                self.teplomer_max = max(self.policka_pomocna.keys())
            for i in range(1, self.teplomer_max):
                # pro vsechny dvojice poli, ktere spolu sousedi v teplomeru udelej vzajemny odkaz a oznac, ktere je vetsi (1)
                if i in self.policka_pomocna.keys() and (i + 1) in self.policka_pomocna.keys():
                    self.policka[self.policka_pomocna[i][0] - 1][self.policka_pomocna[i][1] - 1].append((self.policka_pomocna[i + 1][0], self.policka_pomocna[i + 1][1], -1))
                    self.policka[self.policka_pomocna[i + 1][0] - 1][self.policka_pomocna[i + 1][1] - 1].append((self.policka_pomocna[i][0], self.policka_pomocna[i][1], 1))
        # stejny postup banovani jako v 'Resitel_Nerovnost'
        self.policka_pomocna = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka_pomocna.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka_pomocna[-1].append(0)
        for k in range(1, sudoku.rozmer_cisla + 1):
            for i in range(sudoku.rozmer_mrizka):
                for j in range(sudoku.rozmer_mrizka):
                    if self.policka_pomocna[i][j] == 0:
                        self.flag_hodnota = True
                        for policko in self.policka[i][j]:
                            if policko[2] == 1 and self.policka_pomocna[policko[0] - 1][policko[1] - 1] in [0, k]:
                                self.flag_hodnota = False
                        if self.flag_hodnota:
                            self.policka_pomocna[i][j] = k
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka):
                for k in range(1, self.policka_pomocna[i][j]):
                    sudoku.banuj_policko(i + 1, j + 1, k)
                self.policka_pomocna[i][j] = 0
        for k in range(sudoku.rozmer_cisla, 0, -1):
            for i in range(sudoku.rozmer_mrizka):
                for j in range(sudoku.rozmer_mrizka):
                    if self.policka_pomocna[i][j] == 0:
                        self.flag_hodnota = True
                        for policko in self.policka[i][j]:
                            if policko[2] == -1 and self.policka_pomocna[policko[0] - 1][policko[1] - 1] in [0, k]:
                                self.flag_hodnota = False
                        if self.flag_hodnota:
                            self.policka_pomocna[i][j] = k
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka):
                for k in range(self.policka_pomocna[i][j] + 1, sudoku.rozmer_cisla + 1):
                    sudoku.banuj_policko(i + 1, j + 1, k)
    def napis_zadani(self, sudoku):
        self.policka_pomocna = []
        # nebudeme vypisovat vsechny tabulky teplomer zvlast, ale vypiseme je najednou
        # zpravidla jich mame vice, pokud 2 teplomery maji shodnou banku, coz cloveku ve vizualizaci nevadi
        # nebudeme vypisovat primo cisla, nybrz cislo % 10, aby se nam nerozbila struktura radku
        for i in range(sudoku.rozmer_mrizka):
            self.policka_pomocna.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka_pomocna[-1].append(0)
                for teplomer in sudoku.uloha.mozne_objekty['teplomer']:
                    if sudoku.uloha.objekty[teplomer][i][j] != '':
                        self.policka_pomocna[i][j] = max(self.policka_pomocna[i][j], int(sudoku.uloha.objekty[teplomer][i][j]) % 10)
                if self.policka_pomocna[i][j] == 0:
                    self.policka_pomocna[i][j] = ''
                else:
                    self.policka_pomocna[i][j] = str(self.policka_pomocna[i][j])
        print_policka(self.policka_pomocna, sudoku.rozmer_mrizka, 'Teplomer:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        for policko in self.policka[radek - 1][sloupec - 1]:
            # jsem-li vetsi, banuj u dalsiho policka vetsi cisla
            if policko[2] == 1:
                for i in range(cislo + 1, sudoku.rozmer_cisla + 1):
                    sudoku.banuj_policko(policko[0], policko[1], i)
            # jsem-li mensi, banuj mensi cisla
            else:
                for i in range(1, cislo):
                    sudoku.banuj_policko(policko[0], policko[1], i)

In [19]:
# pravidlo - puntikem jsou vyznaceny vsechny sousedici dvojice policek, ktere se lisi o 1
# vstup - tabulky 9x8 a 8x9, pokud je mezi dvema policky rozdil 1, je vepsano 'X' (podobne jako 'Resitel_Nerovnost')
class Resitel_Sousledna:
    def __init__(self, sudoku):
        self.hodnota_bonus = 1500
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka - 1):
                # maji-li sousedni policka rozdil 1
                if sudoku.uloha.objekty['sousledna_radek'][i][j] == 'X':
                    # zapiseme vzajemny odkaz
                    self.policka[i][j].append((i + 1, j + 2, 1))
                    self.policka[i][j + 1].append((i + 1, j + 1, 1))
                    # vepiseme obrovsky bonus
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i][j + 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j + 1].hodnota_bonus)
                else:
                    # nemaji-li sousedni policka rozdil 1, i presto si dvojici poznacime, ale s jinym indexem (0 misto 1)
                    self.policka[i][j].append((i + 1, j + 2, 0))
                    self.policka[i][j + 1].append((i + 1, j + 1, 0))
        for i in range(sudoku.rozmer_mrizka - 1):
            for j in range(sudoku.rozmer_mrizka):
                if sudoku.uloha.objekty['sousledna_sloupec'][i][j] == 'X':
                    self.policka[i][j].append((i + 2, j + 1, 1))
                    self.policka[i + 1][j].append((i + 1, j + 1, 1))
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i + 1][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i + 1][j].hodnota_bonus)
                else:
                    self.policka[i][j].append((i + 2, j + 1, 0))
                    self.policka[i + 1][j].append((i + 1, j + 1, 0))
    def napis_zadani(self, sudoku):
        print_radky(sudoku.uloha.objekty['sousledna_radek'], sudoku.rozmer_mrizka, 'Sousledna radky:')
        print_sloupce(sudoku.uloha.objekty['sousledna_sloupec'], sudoku.rozmer_mrizka, 'Sousledna sloupce:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        for policko in self.policka[radek - 1][sloupec - 1]:
            # pokud se policka lisi o 1
            if policko[2] == 1:
                # jsem-li 1, soused musi byt 2
                if cislo == 1:
                    sudoku.banuj_policko_inverse(policko[0], policko[1], 2)
                # jsem-li 9, soused musi byt 8
                elif cislo == sudoku.rozmer_cisla:
                    sudoku.banuj_policko_inverse(policko[0], policko[1], cislo - 1)
                # jinak je soused o jedna vetsi ci mensi nez ja
                else:
                    sudoku.banuj_policko_inverse_2(policko[0], policko[1], cislo - 1, cislo + 1)
            # policka se nelisi o 1
            else:
                # jsem-li 1, soused nesmi byt 2
                if cislo == 1:
                    sudoku.banuj_policko(policko[0], policko[1], 2)
                # jsem-li 9, soused nesmi byt 8
                elif cislo == sudoku.rozmer_cisla:
                    sudoku.banuj_policko(policko[0], policko[1], cislo - 1)
                # jinak soused nesmi byt o 1 vetsi ani mensi
                else:
                    sudoku.banuj_policko(policko[0], policko[1], cislo - 1)
                    sudoku.banuj_policko(policko[0], policko[1], cislo + 1)

In [20]:
# pravidlo - symboly 'V' a 'X' jsou oznaceny vsechny dvojice sousedicich poli, jejich soucet je 5 nebo 10
# vstup - tabulky 9x8 a 8x9, pokud je soucet dvou policek 5 nebo 10, je vepsano 'V' nebo 'X' (podobne jako 'Resitel_Nerovnost')
class Resitel_Rimska:
    def __init__(self, sudoku):
        self.hodnota_bonus = 1500
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka - 1):
                # je-li soucet 10
                if sudoku.uloha.objekty['rimska_radek'][i][j] == 'X':
                    # policka si vzajemne odkazeme
                    self.policka[i][j].append((i + 1, j + 2, 10))
                    self.policka[i][j + 1].append((i + 1, j + 1, 10))
                    # cisla nemohou byt 5 (jinak by vedle musela byt take 5)
                    sudoku.banuj_policko(i + 1, j + 1, 5)
                    sudoku.banuj_policko(i + 1, j + 2, 5)
                    # cisla nemohou byt 10 nebo vetsi (teoreticky bychom mohli mit sudoku vetsich rozmeru)
                    for k in range(10, sudoku.rozmer_cisla + 1):
                        sudoku.banuj_policko(i + 1, j + 1, k)
                        sudoku.banuj_policko(i + 1, j + 2, k)
                    # obema polickum pripiseme obrovsky bonus
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i][j + 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j + 1].hodnota_bonus)
                # je-li soucet 5
                elif sudoku.uloha.objekty['rimska_radek'][i][j] == 'V':
                    # policka si vzajemne odkazame
                    self.policka[i][j].append((i + 1, j + 2, 5))
                    self.policka[i][j + 1].append((i + 1, j + 1, 5))
                    # cisla nemohou byt 5 nebo vice
                    for k in range(5, sudoku.rozmer_cisla + 1):
                        sudoku.banuj_policko(i + 1, j + 1, k)
                        sudoku.banuj_policko(i + 1, j + 2, k)
                    # pripiseme bonus
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i][j + 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j + 1].hodnota_bonus)
                # mezi policky neni soucet ani 5 ani 10
                else:
                    # policka si vzajemne odkazeme
                    self.policka[i][j].append((i + 1, j + 2, 0))
                    self.policka[i][j + 1].append((i + 1, j + 1, 0))
        for i in range(sudoku.rozmer_mrizka - 1):
            for j in range(sudoku.rozmer_mrizka):
                if sudoku.uloha.objekty['rimska_sloupec'][i][j] == 'X':
                    self.policka[i][j].append((i + 2, j + 1, 10))
                    self.policka[i + 1][j].append((i + 1, j + 1, 10))
                    sudoku.banuj_policko(i + 1, j + 1, 5)
                    sudoku.banuj_policko(i + 2, j + 1, 5)
                    for k in range(10, sudoku.rozmer_cisla + 1):
                        sudoku.banuj_policko(i + 1, j + 1, k)
                        sudoku.banuj_policko(i + 2, j + 1, k)
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i + 1][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i + 1][j].hodnota_bonus)
                elif sudoku.uloha.objekty['rimska_sloupec'][i][j] == 'V':
                    self.policka[i][j].append((i + 2, j + 1, 5))
                    self.policka[i + 1][j].append((i + 1, j + 1, 5))
                    for k in range(5, sudoku.rozmer_cisla + 1):
                        sudoku.banuj_policko(i + 1, j + 1, k)
                        sudoku.banuj_policko(i + 2, j + 1, k)
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i + 1][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i + 1][j].hodnota_bonus)
                else:
                    self.policka[i][j].append((i + 2, j + 1, 0))
                    self.policka[i + 1][j].append((i + 1, j + 1, 0))
    def napis_zadani(self, sudoku):
        print_radky(sudoku.uloha.objekty['rimska_radek'], sudoku.rozmer_mrizka, 'Rimska radky:')
        print_sloupce(sudoku.uloha.objekty['rimska_sloupec'], sudoku.rozmer_mrizka, 'Rimska sloupce:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        for policko in self.policka[radek - 1][sloupec - 1]:
            # je-li soucet 10
            if policko[2] == 10:
                # soused musi byt doplnek do 10
                sudoku.banuj_policko_inverse(policko[0], policko[1], 10 - cislo)
            # je-li soucet 5
            elif policko[2] == 5:
                # soused musi byt doplnek do 5
                sudoku.banuj_policko_inverse(policko[0], policko[1], 5 - cislo)
            # soucet neni ani 5 ani 10
            else:
                if cislo < 5:
                    # soused nesmi byt doplnek do 5
                    sudoku.banuj_policko(policko[0], policko[1], 5 - cislo)
                if cislo < 10:
                    # soused nesmi byt doplnek do 10
                    sudoku.banuj_policko(policko[0], policko[1], 10 - cislo)

In [21]:
# pravidlo - mezi nekterymi sousedicimi policky je vyznacen jejich rozdil
# vstup - tabulky 9x8 a 8x9 s vepsanymi rozdily mezi nekterymi policky (podobne jako 'Resitel_Nerovnost')
class Resitel_Rozdil:
    def __init__(self, sudoku):
        self.hodnota_bonus = 1500
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka - 1):
                # mezi policky je vepsan rozdil
                if sudoku.uloha.objekty['rozdil_radek'][i][j] != '':
                    # policka si na sebe odkazeme spolu s danym rozdilem
                    self.policka[i][j].append((i + 1, j + 2, int(sudoku.uloha.objekty['rozdil_radek'][i][j])))
                    self.policka[i][j + 1].append((i + 1, j + 1, int(sudoku.uloha.objekty['rozdil_radek'][i][j])))
                    # je-li rozdil prilis velky, pak jiz nyni muzeme rici, ze policka nemohou obsahovat nektera cisla
                    # je-li napr. rozdil 6, pak cisla not in [4,5,6]
                    if 2 * int(sudoku.uloha.objekty['rozdil_radek'][i][j]) > sudoku.rozmer_cisla:
                        for k in range(sudoku.rozmer_cisla - int(sudoku.uloha.objekty['rozdil_radek'][i][j]) + 1, int(sudoku.uloha.objekty['rozdil_radek'][i][j]) + 1):
                            sudoku.banuj_policko(i + 1, j + 1, k)
                            sudoku.banuj_policko(i + 1, j + 2, k)
                    # pripiseme polickum obrovsky bonus
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i][j + 1].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j + 1].hodnota_bonus)
        for i in range(sudoku.rozmer_mrizka - 1):
            for j in range(sudoku.rozmer_mrizka):
                if sudoku.uloha.objekty['rozdil_sloupec'][i][j] != '':
                    self.policka[i][j].append((i + 2, j + 1, int(sudoku.uloha.objekty['rozdil_sloupec'][i][j])))
                    self.policka[i + 1][j].append((i + 1, j + 1, int(sudoku.uloha.objekty['rozdil_sloupec'][i][j])))
                    if 2 * int(sudoku.uloha.objekty['rozdil_sloupec'][i][j]) > sudoku.rozmer_cisla:
                        for k in range(sudoku.rozmer_cisla - int(sudoku.uloha.objekty['rozdil_sloupec'][i][j]) + 1, int(sudoku.uloha.objekty['rozdil_sloupec'][i][j]) + 1):
                            sudoku.banuj_policko(i + 1, j + 1, k)
                            sudoku.banuj_policko(i + 2, j + 1, k)
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                    sudoku.policka[i + 1][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i + 1][j].hodnota_bonus)
    def napis_zadani(self, sudoku):
        print_radky(sudoku.uloha.objekty['rozdil_radek'], sudoku.rozmer_mrizka, 'Rozdil radky:')
        print_sloupce(sudoku.uloha.objekty['rozdil_sloupec'], sudoku.rozmer_mrizka, 'Rozdil sloupce:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        for policko in self.policka[radek - 1][sloupec - 1]:
            # je-li vepsane cislo mensi nez rozdil, pak soused musi byt roven danemu cislu plus rozdil
            if cislo < policko[2]:
                sudoku.banuj_policko_inverse(policko[0], policko[1], cislo + policko[2])
            # je-li vepsane cislo s rozdilem vice, nez rozmer sudoku, pak soused musi byt roven danemu cislu minus rozdil
            elif cislo + policko[2] > sudoku.rozmer_cisla:
                sudoku.banuj_policko_inverse(policko[0], policko[1], cislo - policko[2])
            # jinak muze but soused dane cislo plus minus rozdil
            else:
                sudoku.banuj_policko_inverse_2(policko[0], policko[1], cislo - policko[2], cislo + policko[2])

In [22]:
# pravidlo - na nekterych polich jsou sipky:
    # tyto sipky udavaji kterym smerem se v danem radku ci sloupci nachazi cislo 9
    # cisla na miste techto sipek zaroven udavaji, o kolik poli je 9 od sipky vzdalena
# vstup - tabulka 9x9, obsahuje sipky '<', '>', 'V', 'A'
class Resitel_KdeJe9:
    def __init__(self, sudoku):
        self.hodnota_bonus = 1500
        self.policka = []
        for i in range(sudoku.rozmer_mrizka):
            self.policka.append([])
            for j in range(sudoku.rozmer_mrizka):
                self.policka[-1].append([])
        for i in range(sudoku.rozmer_mrizka):
            for j in range(sudoku.rozmer_mrizka):
                # pokud narazim na sipku nahoru
                if sudoku.uloha.objekty['kdeje9'][i][j] == 'A':
                    # poznacim si, ze ja sam obsahuji sipku
                    self.policka[i][j].append((i + 1, j + 1, 'A'))
                    # polickum ve smeru sipky zapisi odkaz, kde se sipka nachazi a jak je daleko
                    for k in range(i):
                        self.policka[k][j].append((i + 1, j + 1, i - k))
                    # na policku se sipkou nemuze byt prilis velike cislo, abychom nevybocili ven z tabulky
                    for k in range(i + 1, sudoku.rozmer_cisla + 1):
                        sudoku.banuj_policko(i + 1, j + 1, k)
                    # policka na opacnou stranu od sipky nemohou obsahovat cislo 9
                    for k in range(i + 1, sudoku.rozmer_mrizka):
                        sudoku.banuj_policko(k + 1, j + 1, 9)
                    # policku s sipkou pripisi obrovsky bonus
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                elif sudoku.uloha.objekty['kdeje9'][i][j] == 'V':
                    self.policka[i][j].append((i + 1, j + 1, 'V'))
                    for k in range(i + 1, sudoku.rozmer_mrizka):
                        self.policka[k][j].append((i + 1, j + 1, k - i))
                    for k in range(sudoku.rozmer_mrizka - i, sudoku.rozmer_cisla + 1):
                        sudoku.banuj_policko(i + 1, j + 1, k)
                    for k in range(i):
                        sudoku.banuj_policko(k + 1, j + 1, 9)
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                elif sudoku.uloha.objekty['kdeje9'][i][j] == '<':
                    self.policka[i][j].append((i + 1, j + 1, '<'))
                    for k in range(j):
                        self.policka[i][k].append((i + 1, j + 1, j - k))
                    for k in range(j + 1, sudoku.rozmer_cisla + 1):
                        sudoku.banuj_policko(i + 1, j + 1, k)
                    for k in range(j + 1, sudoku.rozmer_mrizka):
                        sudoku.banuj_policko(i + 1, k + 1, 9)
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
                elif sudoku.uloha.objekty['kdeje9'][i][j] == '>':
                    self.policka[i][j].append((i + 1, j + 1, '>'))
                    for k in range(j + 1, sudoku.rozmer_mrizka):
                        self.policka[i][k].append((i + 1, j + 1, k - j))
                    for k in range(sudoku.rozmer_mrizka - j, sudoku.rozmer_cisla + 1):
                        sudoku.banuj_policko(i + 1, j + 1, k)
                    for k in range(j):
                        sudoku.banuj_policko(i + 1, k + 1, 9)
                    sudoku.policka[i][j].hodnota_bonus = max(self.hodnota_bonus, sudoku.policka[i][j].hodnota_bonus)
    def napis_zadani(self, sudoku):
        print_policka(sudoku.uloha.objekty['kdeje9'], sudoku.rozmer_mrizka, 'Kde je 9:')
    def banuj(self, sudoku, radek, sloupec, cislo):
        for policko in self.policka[radek - 1][sloupec - 1]:
            # jedna-li se o policko se sipkou nahoru
            if policko[2] == 'A':
                # na pole, kam se odkazuje sipka muze byt pouze 9
                sudoku.banuj_policko_inverse(radek - cislo, sloupec, 9)
            elif policko[2] == 'V':
                sudoku.banuj_policko_inverse(radek + cislo, sloupec, 9)
            elif policko[2] == '<':
                sudoku.banuj_policko_inverse(radek, sloupec - cislo, 9)
            elif policko[2] == '>':
                sudoku.banuj_policko_inverse(radek, sloupec + cislo, 9)
            # nejsem-li 9 a odkazuje-li se na mne nejaka sipka, musim ji zakazat cislo s poctem kroku ke mne
            elif cislo != 9:
                sudoku.banuj_policko(policko[0], policko[1], policko[2])
            # jsem-li 9 a odkazuje-li se na mne nejaka sipka, pak musi obsahovat pocet kroku ke mne
            elif cislo == 9:
                sudoku.banuj_policko_inverse(policko[0], policko[1], policko[2])

In [23]:
# vzor pro budouci dalsi tridy
class Resitel:
    def __init__(self, sudoku):
        pass
    def napis_zadani(self, sudoku):
        pass
    def banuj(self, sudoku, radek, sloupec, cislo):
        pass

In [24]:
class Sudoku:
    def __init__(self, uloha, zarazka_strop = 10**5):
        self.zarazka = 0
        self.zarazka_strop = zarazka_strop
        self.rozmer_cisla = uloha.rozmer_cisla
        self.rozmer_mrizka = uloha.rozmer_mrizka
        self.uloha = copy.deepcopy(uloha)
        self.policka = []
        for i in range(uloha.rozmer_mrizka):
            self.policka.append([])
            for j in range(uloha.rozmer_mrizka):
                self.policka[-1].append(Policko(self.rozmer_cisla))
        self.kroky = []
        self.resitele = []
        self.dalsi = Dalsi()
        self.hodnota_max = 10**6
        self.hodnota_cislo = 10**3
        self.flag_resime = False
        self.pocet_reseni = 0
        # pri iniciaci si tato trida dle vstupu (trida 'Uloha') vytvori potrebne tridy 'Resitel_SpecialniTyp'
        self.nacti_resitele()
        # do pole 'policka' si zapise cisla ze zadani
        self.nacti_cisla()
        # ohodnoti si jednotliva pole, aby vedela, kde zacit resit, a urci pole, do ktereho zacne zapisovat
        self.ohodnot_vse()
        self.urci_dalsi_pole()
        self.urci_dalsi_krok()
    def nacti_resitele(self):
        self.resitele.append(Resitel_Sudoku(self))
        if 'basic' in self.uloha.typ:
            self.resitele.append(Resitel_Basic(self))
        if 'diagonal' in self.uloha.typ:
            self.resitele.append(Resitel_Diagonal(self))
        if 'windoku' in self.uloha.typ:
            self.resitele.append(Resitel_Windoku(self))
        if 'region' in self.uloha.typ:
            self.resitele.append(Resitel_Region(self))
        if 'suda' in self.uloha.typ:
            self.resitele.append(Resitel_Suda(self))
        if 'licha' in self.uloha.typ:
            self.resitele.append(Resitel_Licha(self))
        if 'antijezdec' in self.uloha.typ:
            self.resitele.append(Resitel_AntiJezdec(self))
        if 'pevnost' in self.uloha.typ:
            self.resitele.append(Resitel_Pevnost(self))
        if 'nerovnost' in self.uloha.typ:
            self.resitele.append(Resitel_Nerovnost(self))
        if 'teplomer' in self.uloha.typ:
            self.resitele.append(Resitel_Teplomer(self))
        if 'sousledna' in self.uloha.typ:
            self.resitele.append(Resitel_Sousledna(self))
        if 'rimska' in self.uloha.typ:
            self.resitele.append(Resitel_Rimska(self))
        if 'rozdil' in self.uloha.typ:
            self.resitele.append(Resitel_Rozdil(self))
        if 'kdeje9' in self.uloha.typ:
            self.resitele.append(Resitel_KdeJe9(self))
    def nacti_cisla(self):
        for i in range(self.rozmer_mrizka):
            for j in range(self.rozmer_mrizka):
                # pokud zde mame cislo
                if self.uloha.objekty['cisla'][i][j] != '':
                    self.policka[i][j].cislo = int(self.uloha.objekty['cisla'][i][j])
                    # je nutne uz zde myslet na banovani
                    self.banuj(i + 1, j + 1, self.policka[i][j].cislo)
    def ohodnot(self, policko):
        # logika je takova, ze cim vyssi hodnota u policka, tim driv prijde na radu v ramci vyplnovani
        # pokud je jiz vyplneno, dostane 0
        if policko.cislo != 0:
            policko.hodnota = 0
        # neni-li vyplneno, ale nelze-li do nej zapsat zadne cislo, nastavime maximalni hodnotu
        elif policko.bany_pocet == self.rozmer_cisla:
            policko.hodnota = self.hodnota_max
        # jinak priradime hodnotu dle poctu banu a bonusu
        else:
            policko.hodnota = self.hodnota_cislo * policko.bany_pocet + policko.hodnota_bonus
    def ohodnot_vse(self):
        for i in range(self.rozmer_mrizka):
            for j in range(self.rozmer_mrizka):
                self.ohodnot(self.policka[i][j])
    def urci_dalsi_pole(self):
        # urci, do jakeho policka budeme zapisovat (najde policko s nejvyssi hodnotou)
        self.dalsi.hodnota = 0
        for i in range(self.rozmer_mrizka):
            for j in range(self.rozmer_mrizka):
                if self.policka[i][j].hodnota > self.dalsi.hodnota:
                    self.dalsi.radek = i + 1
                    self.dalsi.sloupec = j + 1
                    self.dalsi.hodnota = self.policka[i][j].hodnota
    # tato funkce by se s trochou nadsazky dala prohlasit za jadro logiky programu
    # nastavuje indikator, ktery udava dalsi chovani programu:
        # 2 - sudoku je kompletne vyplneno
        # 1 - v dalsim kroku budeme zapisovat cislo
        # 0 - v dalsim kroku budeme menit posledne napsane cislo
        # -1 - v dalsim kroku budeme mazat posledni napsane cislo
        # -2 - prosli jsme vsechny moznosti = konec programu
    def urci_dalsi_krok(self):
        # pokud predchozi krok byl 'sudoku vyplneno'
        if self.dalsi.indikator == 2:
            # pokud posledni zapsane cislo mohu zmenit na jine
            if self.kroky[-1].cislo_poradi + 1 < len(self.kroky[-1].mozna_cisla):
                self.dalsi.indikator = 0
            # jinak smaz posledne zapsane cislo
            else:
                self.dalsi.indikator = -1
        # pokud predchozi krok byl zapis nebo zmena cisla
        elif self.dalsi.indikator in [0, 1]:
            # pokud nedoslo k erroru a nejvyssi hodnota policka je 0 (tedy vsechna vyplnena)
            if (not self.dalsi.error) and self.dalsi.hodnota == 0:
                self.dalsi.indikator = 2
            # pokud nedoslo k erroru a nejvyssi hodnota neni max (max hodnota znamena, ze do nejakeho policka nelze zadne cislo zapsat)
            elif (not self.dalsi.error) and self.dalsi.hodnota < self.hodnota_max:
                self.dalsi.indikator = 1
            # pokud posledni zapsane cislo mohu zmenit na jine
            elif self.kroky[-1].cislo_poradi + 1 < len(self.kroky[-1].mozna_cisla):
                self.dalsi.indikator = 0
            # jinak budeme mazat
            else:
                self.dalsi.indikator = -1
        # pokud predchozi krok byl mazani (dalsi krok nesmi byt zapsani cisla, jinak se cyklime)
        elif self.dalsi.indikator == -1:
            # pokud jsem na kroku 0 (tedy na pocatku lusteni nebo jsem se vratil na zacatek)
            if len(self.kroky) == 0:
                self.dalsi.indikator = -2
            # pokud posledni zapsane cislo lze zmenit na jine
            elif self.kroky[-1].cislo_poradi + 1 < len(self.kroky[-1].mozna_cisla):
                self.dalsi.indikator = 0
            # jinak mazeme
            else:
                self.dalsi.indikator = -1
        # zresetuj error
        self.dalsi.error = False
    def udelej_dalsi_krok(self):
        if self.dalsi.indikator == 2:
            self.napis_reseni()
        elif self.dalsi.indikator == 1:
            self.napis_cislo()
        elif self.dalsi.indikator == 0:
            self.zmen_cislo()
        elif self.dalsi.indikator == -1:
            self.smaz_cislo()
        self.urci_dalsi_pole()
        self.urci_dalsi_krok()
    def napis_cislo(self):
        # vytvorime novou polozku listu 'kroky' pro trasovani postupu
        self.kroky.append(Krok(self.dalsi.radek, self.dalsi.sloupec))
        # pro dane policko (kam budeme zapisovat) si vypiseme dle banu pripustna cisla
        for i in range(self.rozmer_cisla):
            if not self.policka[self.dalsi.radek - 1][self.dalsi.sloupec - 1].bany[i]:
                self.kroky[-1].mozna_cisla.append(i + 1)
        # napiseme prvni z moznych cisel
        self.policka[self.dalsi.radek - 1][self.dalsi.sloupec - 1].cislo = self.kroky[-1].mozna_cisla[0]
        # spustime banovaci proces, abychom na zaklade vepsaneho cisla poznacili, kam nelze ktera cisla psat
        self.banuj(self.dalsi.radek, self.dalsi.sloupec, self.kroky[-1].mozna_cisla[0])
    def zmen_cislo(self):
        # zrusime vsechny bany, ktere vznikly v kroku zapsani cisla, ktere budeme menit
        self.odbanuj()
        self.kroky[-1].cislo_poradi += 1
        # zapiseme nove cislo
        self.policka[self.kroky[-1].radek - 1][self.kroky[-1].sloupec - 1].cislo = self.kroky[-1].mozna_cisla[self.kroky[-1].cislo_poradi]
        # spustime banovaci proces
        self.banuj(self.kroky[-1].radek, self.kroky[-1].sloupec, self.kroky[-1].mozna_cisla[self.kroky[-1].cislo_poradi])
    def smaz_cislo(self):
        # nastavime cislo na 0
        self.policka[self.kroky[-1].radek - 1][self.kroky[-1].sloupec - 1].cislo = 0
        # zrusime bany
        self.odbanuj()
        # smazeme posledni krok
        self.kroky.pop()
    def banuj(self, radek, sloupec, cislo):
        # postupne zavolam jednotlive tridy typu 'Resitel_SpecialniTyp'
        for resitel in self.resitele:
            resitel.banuj(self, radek, sloupec, cislo)
    # tato funkce je volana funkcemi 'banuj' ze trid 'Resitel_SpecialniTyp'
    def banuj_policko(self, radek, sloupec, cislo):
        # pokud ban jeste neexistuje
        if not self.policka[radek - 1][sloupec - 1].bany[cislo - 1]:
            # zapis ban
            self.policka[radek - 1][sloupec - 1].bany[cislo - 1] = True
            self.policka[radek - 1][sloupec - 1].bany_pocet += 1
            # znovu ohodnot policko (jeho hodnota se zvysi pri vyssim poctu banu)
            self.ohodnot(self.policka[radek - 1][sloupec - 1])
            # pokud jsme ve fazi reseni (nikoli ve fazi nacitani zadani)
            if self.flag_resime:
                # zapis ban do daneho kroku, abychom mohli pozdeji odbanovat
                self.kroky[-1].bany.append(Krok_Ban(radek, sloupec, cislo))
    # tato funkce je volana funkcemi 'banuj' ze trid 'Resitel_SpecialniTyp'
    # jedna se o opak funkce 'banuj_policko', pro zadane cislo zabanuje z policka vsechna ostatni cisla
    def banuj_policko_inverse(self, radek, sloupec, cislo):
        for i in range(self.rozmer_cisla):
            if (not self.policka[radek - 1][sloupec - 1].bany[i]) and i + 1 != cislo:
                self.policka[radek - 1][sloupec - 1].bany[i] = True
                self.policka[radek - 1][sloupec - 1].bany_pocet += 1
                if self.flag_resime:
                    self.kroky[-1].bany.append(Krok_Ban(radek, sloupec, i + 1))
        self.ohodnot(self.policka[radek - 1][sloupec - 1])
    # tato funkce je volana funkcemi 'banuj' ze trid 'Resitel_SpecialniTyp'
    # jedna se o obdobu funkce 'banuj_policko_inverse', ale banuje vse vyjma dvou zadanych cisel
    def banuj_policko_inverse_2(self, radek, sloupec, cislo_1, cislo_2):
        for i in range(self.rozmer_cisla):
            if (not self.policka[radek - 1][sloupec - 1].bany[i]) and i + 1 != cislo_1 and i + 1 != cislo_2:
                self.policka[radek - 1][sloupec - 1].bany[i] = True
                self.policka[radek - 1][sloupec - 1].bany_pocet += 1
                if self.flag_resime:
                    self.kroky[-1].bany.append(Krok_Ban(radek, sloupec, i + 1))
        self.ohodnot(self.policka[radek - 1][sloupec - 1])
    def odbanuj(self):
        # projde vsechny bany udelane v poslednim kroku
        for unban in self.kroky[-1].bany:
            self.policka[unban.radek - 1][unban.sloupec - 1].bany[unban.cislo - 1] = False
            self.policka[unban.radek - 1][unban.sloupec - 1].bany_pocet -= 1
            self.ohodnot(self.policka[unban.radek - 1][unban.sloupec - 1])
        # smaze frontu banu z posledniho kroku
        self.kroky[-1].bany = []
    def napis_zadani(self):
        # zavola vsechny resitele, aby si vypsali sve kusy zadani
        for resitel in self.resitele:
            resitel.napis_zadani(self)
    def napis_reseni(self):
        print('Sudoku reseni (pocet kroku {0})'.format(self.zarazka))
        for i in range(self.rozmer_mrizka):
            pomocny_string = ''
            if i % 3 == 0:
                print('-------------')
            for j in range(self.rozmer_mrizka):
                if j % 3 == 0:
                    pomocny_string += '|'
                if self.policka[i][j].cislo == 0:
                    pomocny_string += ' '
                else:
                    pomocny_string += str(self.policka[i][j].cislo)
            pomocny_string += '|'
            print(pomocny_string)
        print('-------------')
        print()
        self.pocet_reseni += 1
    # nepouziva se pro beh programu, ale pro potreby debugovani
    def napis_bany(self, cislo):
        print('Sudoku reseni (pocet kroku {0})'.format(self.zarazka))
        for i in range(self.rozmer_mrizka):
            pomocny_string = ''
            if i % 3 == 0:
                print('-------------')
            for j in range(self.rozmer_mrizka):
                if j % 3 == 0:
                    pomocny_string += '|'
                if self.policka[i][j].cislo == cislo:
                    pomocny_string += str(cislo)
                elif self.policka[i][j].bany[cislo - 1]:
                    pomocny_string += 'X'
                else:
                    pomocny_string += ' '
            pomocny_string += '|'
            print(pomocny_string)
        print('-------------')
    # hlavni funkce
    def vyres_sudoku(self):
        self.flag_resime = True
        self.napis_zadani()
        # vyresi sudoku
        while self.zarazka < self.zarazka_strop and self.dalsi.indikator != -2:
            self.udelej_dalsi_krok()
            self.zarazka += 1
        # vypise finalni komentar
        if self.pocet_reseni == 0:
            if self.zarazka == self.zarazka_strop:
                print('Je nam lito, ale sudoku se na {0} kroku nepodarilo vylustit...'.format(self.zarazka))
            else:
                print('Je nam lito, ale toto sudoku nema reseni, provedeno {0} kroku'.format(self.zarazka))
        else:
            if self.zarazka == self.zarazka_strop:
                print('Reseni nalezena na {0} kroku uvedena vyse, mohou existovat dalsi...'.format(self.zarazka))
            else:
                print('Vsechna reseni byla na {0} kroku nalezena...'.format(self.zarazka))
    # nepouziva se pro beh programu, ale pro potreby debugovani
    def vyres_sudoku_pomalu(self, krokuj_od, pocet_kroku, velikost_kroku):
        self.flag_resime = True
        self.napis_reseni()
        self.urci_dalsi_krok()
        while self.zarazka < krokuj_od + pocet_kroku:
            self.udelej_dalsi_krok()
            self.zarazka += 1
            if self.zarazka >= krokuj_od and (self.zarazka - krokuj_od) % velikost_kroku == 0:
                self.napis_reseni()

In [50]:
U = Uloha('sudoku_kdeje9.csv')
S = Sudoku(U)
S.vyres_sudoku()

Sudoku zadani:
Typ: basic kdeje9        
-------------
|   |   |  2|
| 7 |   |   |
|   | 3 |   |
-------------
|   |  6|   |
|  5|   |   |
|   |8  |   |
-------------
|   |   |   |
|   |   |   |
|4  |   |   |
-------------

Kde je 9:
-------------
| V | V | < |
|>  |  V|VV |
|   |A  |   |
-------------
|  <| V |   |
|>  |>  |   |
| > |   |<  |
-------------
| > |  A|A  |
|A> |   |  A|
|   |   | < |
-------------

Sudoku reseni (pocet kroku 157)
-------------
|136|985|742|
|872|164|359|
|954|237|618|
-------------
|791|456|823|
|685|312|974|
|243|879|165|
-------------
|368|541|297|
|519|723|486|
|427|698|531|
-------------

Vsechna reseni byla na 244 kroku nalezena...
